In [148]:
import gEconpy as ge
import gEconpy.plotting as gp

# gEconpy preliminaries

## GCN file

I've already written the model into a GCN file and saved it. Here is the file.

In [149]:
with open('rbc.gcn', 'r') as file:
    print(file.read())

assumptions
{
	positive
	{
		K[], A[], mc[], r[], w[], Y[], alpha;
	};
};

block HOUSEHOLD
{
	definitions
	{
		u[] = gamma * log(C[]) + psi * log(L[]);
	};

	objective
	{
		U[] = u[] + beta * E[][U[1]];
	};

	controls
	{
		C[], I[], K[], L[], N[];
	};

	constraints
	{
		C[] + I[] = w[] * N[] + r[] * K[-1] + Div[] : lambda[];
		N[] + L[] = 1;
		K[] = (1 - delta) * K[-1] + I[];
	};

	calibration
	{
		beta = 0.98;
		delta = 0.035;
		gamma = 1;
		N[ss] = 0.33 -> psi;
	};
};

block FIRM
{
	objective
	{
		TC[] = -(w[] * N[] + r[] * K[-1]);
	};

	controls
	{
		N[], K[-1];
	};

	constraints
	{
		Y[] = A[] * K[-1] ^ alpha * N[] ^ (1 - alpha): mc[];
	};

	identities
	{
		mc[] = 1;
		Div[] = Y[] + TC[];
	};
	
	calibration
	{
		alpha = 0.33;
	};
};

block EXOGENOUS
{
	identities
	{
		log(A[]) = rho * log(A[-1]) + epsilon[];
	};

	shocks
	{
		epsilon[];
	};

	calibration
	{
		rho = 0.95;
	};
};



## Load the model

To better see how things work under the hood, I am disabling automatic symbolic simplification. This is never actually recommended.

In [150]:
mod = ge.gEconModel('rbc.gcn', 
                    simplify_tryreduce=False, 
                    simplify_blocks=False,
                    simplify_constants=False)

Model Building Complete.
Found:
	16 equations
	16 variables
	1 stochastic shock
		 0 / 1 has a defined prior. 
	5 parameter
		 0 / 5 has a defined prior. 
	1 calibrating equation
	1 parameter to calibrate
 Model appears well defined and ready to proceed to solving.



# Questions

## Household First Order Conditions

gEconpy derives first order conditions automatically. We can access these block-by-block by looking into the `mod.blocks` property. The first 3 equations are the model constraints, then the objective, then the derivatives of the block lagrangian, in the order they were entered by the user.

In [151]:
import sympy as sp
L = sp.Symbol('\mathcal L')
household_controls = mod.blocks['HOUSEHOLD'].controls
household_equations = mod.blocks['HOUSEHOLD'].system_equations
for control, eq in zip(household_controls, household_equations[-len(household_controls):]):
    display(sp.Eq(sp.diff(L, control, evaluate=False), eq))

Eq(Derivative(\mathcal L, C_t), gamma/C_t - lambda_t)

Eq(Derivative(\mathcal L, I_t), lambda__H_2_t - lambda_t)

Eq(Derivative(\mathcal L, K_t), beta*(-lambda__H_2_t+1*(delta - 1) + lambda_t+1*r_t+1) - lambda__H_2_t)

Eq(Derivative(\mathcal L, L_t), psi/L_t - lambda__H_1_t)

Eq(Derivative(\mathcal L, N_t), -lambda__H_1_t + lambda_t*w_t)

In [152]:
dL_dC, dL_dI, dL_dK, dL_dL, dL_dN = household_equations[4:]

To get the desired equations, we need to do a bit of manipulation. This is easy to do because all of these expressions are sympy expressions, allowing for fast symbolic solving/substitution. 

First, we need to add the model variables to the notebook global namespace so we can refer to them.

In [153]:
all_vars = [v for x in mod.variables for v in [x.step_backward(), x, x.step_forward(), x.to_ss()]]
var_dict = {}
for x in all_vars:
    var_dict[x.safe_name] = x
for x in mod.free_param_dict.to_sympy().keys():
    var_dict[x.name] = x
    
globals().update(var_dict)

### Euler Equation

In [154]:
from gEconpy.shared.utilities import step_equation_forward, step_equation_backward, eq_to_ss
lam = sp.solve(dL_dC, lambda_t)[0]
lam2 = sp.solve(dL_dI.subs({lambda_t:lam}), lambda__H_2_t)[0]
euler = dL_dK.subs({lambda__H_2_t:lam2, 
                    lambda__H_2_tp1:step_equation_forward(lam2),
                    lambda_tp1:step_equation_forward(lam2)})
euler = sp.Eq(C_tp1 / C_t, sp.solve(euler, C_tp1)[0] / C_t)

In [155]:
euler

Eq(C_t+1/C_t, beta*(-delta + r_t+1 + 1))

### Labor supply curve

In [156]:
lam1 = sp.solve(dL_dN, lambda__H_1_t)[0]
wage = sp.solve(dL_dL.subs({lambda__H_1_t:lam1, lambda_t:lam}), w_t)[0]
sp.Eq(w_t, wage)

Eq(w_t, psi*C_t/(gamma*L_t))

## Firm FOC

In the exact same way, we can look into the firm FoC.

In [157]:
firm_controls = mod.blocks['FIRM'].controls
firm_equations = mod.blocks['FIRM'].system_equations
for control, eq in zip(firm_controls, firm_equations[-len(controls):]):
    display(sp.Eq(L.diff(control, evaluate=False), eq))

Eq(Derivative(\mathcal L, N_t), A_t*K_t-1**alpha*mc_t*(1 - alpha)/N_t**alpha - w_t)

Eq(Derivative(\mathcal L, K_t-1), alpha*A_t*K_t-1**(alpha - 1)*N_t**(1 - alpha)*mc_t - r_t)

The only difference with the reference answers is the inclusion of the marginal cost term, which is normalized to 1 in perfect competition. 

In [158]:
sp.Eq(w_t, sp.solve(firm_equations[-2], w_t)[0])

Eq(w_t, A_t*K_t-1**alpha*mc_t*(1 - alpha)/N_t**alpha)

In [159]:
sp.Eq(r_t, sp.solve(firm_equations[-1], r_t)[0])

Eq(r_t, alpha*A_t*K_t-1**(alpha - 1)*N_t**(1 - alpha)*mc_t)

## Implicit Market Clearing

This is easily shown by substituting the definition of total costs into the definition dividends, then putting the resulting term into the household budget constraint. 

In [160]:
household_budget = household_equations[0]
dividends = sp.solve(firm_equations[1], Div_t)[0]
total_cost = sp.solve(firm_equations[3], TC_t)[0]

household_budget.subs({Div_t:dividends,
                       TC_t:total_cost}).simplify()

-C_t - I_t + Y_t

## Derive the steady state

Step 1: Get factor prices

In [226]:
# r_ss is easy, it fall straight out of the Euler equation.
r_ss_solved= sp.solve(eq_to_ss(euler), r_ss)[0]

# for w_ss, plug factor demands back into the production function,
# then solve for marginal cost
am1 = sp.Symbol(r'\alpha^{-1}', positive=True, real=True)

capital_demand = sp.solve(firm_equations[-1]
     .replace(K_tm1 ** (alpha - 1), K_tm1 ** alpha / K_tm1)
     .subs(production_func + Y_t, Y_t), K_tm1)[0]

labor_demand = sp.solve(firm_equations[-2]
     .replace(N_t ** -alpha, N_t ** (1 - alpha) / N_t)
     .subs(production_func + Y_t, Y_t), N_t)[0]

production_func = mod.blocks['FIRM'].system_equations[2]
sp.solve(production_func.subs({K_tm1:capital_demand,
                      N_t:labor_demand,
                      A_t:1}), mc_t)[0]

-r_t**alpha*w_t**(1 - alpha)*exp(alpha*log(-(alpha - 1)/alpha))/(alpha - 1)

In [215]:
capital_demand

alpha*Y_t*mc_t/r_t

In [196]:
sp.solve(firm_equations[-1], K_tm1)[0]

(N_t**(alpha - 1)*r_t/(alpha*A_t*mc_t))**(1/(alpha - 1))

In [193]:
production_func + Y_t

A_t*K_t-1**alpha*N_t**(1 - alpha)

In [78]:
mod.steady_state()
mod.solve_model(not_loglin_variable=['Div'])

Steady state found! Sum of squared residuals is 5.516578011665664e-25
Solution found, sum of squared residuals:  3.299074422098197e-29
Norm of deterministic part: 0.000000000
Norm of stochastic part:    0.000000000


In [8]:
mod.print_steady_state()

A_ss                    1.000
C_ss                    0.629
Div_ss                  0.000
I_ss                    0.166
K_ss                    4.733
L_ss                    0.670
N_ss                    0.330
TC_ss                  -0.795
U_ss                  -57.590
Y_ss                    0.795
lambda__H_1_ss          2.565
lambda_ss               1.590
r_ss                    0.055
w_ss                    1.613


In addition, the following parameter values were calibrated:
psi                     1.719
